In [10]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records
import models
models_path = models.__path__[0]
import os

fix_block_number = 41354245
fix_block_number = 41323403

fix_block_number = 41323403 - 1

fix_block_number = 41454653 + 1000

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 137,
}

context, _model_loader = create_cmf(cmf_param)

fix_block_number = context.block_number

In [11]:
from models.credmark.protocols.ichi.vaults import ICHI_POLYGON_COINS, ICHI_POLYGON_VAULTS

In [12]:
if False:
    for coin in ICHI_POLYGON_COINS:
        tok = Token(coin).as_erc20(set_loaded=True)
        print((tok.symbol, tok.name, tok.decimals))

In [13]:
from models.tmp_abi_lookup import ICHI_VAULT, ICHI_VAULT_FACTORY, UNISWAP_V3_POOL_ABI

In [14]:
vault_factory = Contract('0x2d2c72C4dC71AA32D64e5142e336741131A73fc0').set_abi(ICHI_VAULT_FACTORY, set_loaded=True)
if False:
    vault_created = pd.DataFrame(vault_factory.fetch_events(vault_factory.events.ICHIVaultCreated, from_block=25_697_834, by_range=10_000))
    # vault_created = pd.DataFrame(vault_factory.fetch_events(vault_factory.events.ICHIVaultCreated, from_block=0))
    vault_created.to_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))
else:
    vault_created = pd.read_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))

# vault_created.ichiVault.isin(['0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81'])
# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in vault_created.ichiVault.to_list()

# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in ICHI_POLYGON_VAULTS
assert len(set(ICHI_POLYGON_VAULTS)) == len(set(vault_created.ichiVault))

# context.run_model('token.deployment', {'address': '0x2d2c72C4dC71AA32D64e5142e336741131A73fc0'})

In [15]:
if False:
    for n_vault, vault in enumerate(ICHI_POLYGON_VAULTS):
       vault = Contract(vault).set_abi(abi=ICHI_VAULT, set_loaded=True)
       token0 = Token(vault.functions.token0().call()).as_erc20(set_loaded=True)
       token1 = Token(vault.functions.token1().call()).as_erc20(set_loaded=True)

       print((n_vault, vault,
              vault.functions.owner().call(), vault.functions.pool().call(),
              token0.symbol,  token1.symbol,
              vault.functions.allowToken0().call(), vault.functions.allowToken1().call(),
              vault.functions.totalSupply().call() / (10**vault.functions.decimals().call())))
       
       # print((vault, vault.functions.pool().call()))

In [16]:
# allowToken0, allowToken1, balanceOf, getTotalAmounts, totalSupply, token0, token1, symbol, name41354245
# getBasePosition, getLimitPosition, hysteresis, 
# owner, ichiVaultFactory, pool, tickSpacing, fee

- vault operations
- withdraw, transfer
- deposit, transfer
- rebalance

- 0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7
- 0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0xED26b64F351A04Acbf16AA3782969e4a53B12c54', 'WETH', 'DPI', False, True, 280.73224104767195

all transfers

In [36]:
vault_addr = Address('0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7')
vault_ichi = Token(vault_addr).set_abi(abi=ICHI_VAULT, set_loaded=True)
vault_pool = Contract(Address(vault_ichi.functions.pool().call())).set_abi(UNISWAP_V3_POOL_ABI, set_loaded=True)

affiliate = Address(vault_ichi.functions.affiliate().call())
ichi_vault_factory = Contract(vault_ichi.functions.ichiVaultFactory().call()).set_abi(abi=ICHI_VAULT_FACTORY, set_loaded=True)
fee_recipient = Address(ichi_vault_factory.functions.feeRecipient().call())
baseFee = ichi_vault_factory.functions.baseFee().call()
baseFeeSplit = ichi_vault_factory.functions.baseFeeSplit().call()

token0 = Token(vault_ichi.functions.token0().call()).as_erc20(set_loaded=True)
token1 = Token(vault_ichi.functions.token1().call()).as_erc20(set_loaded=True)

allow_token0 = vault_ichi.functions.allowToken0().call()
allow_token1 = vault_ichi.functions.allowToken1().call()

assert not (allow_token0 and allow_token1) and (allow_token0 or allow_token1)

(vault_addr, vault_pool, affiliate, fee_recipient, allow_token0, allow_token1, baseFee / 1e18, baseFeeSplit / 1e18)

('0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7',
 Contract(address='0xed26b64f351a04acbf16aa3782969e4a53b12c54'),
 '0x3d1e6349f508199fdcaf339c1c351dff1d5fd072',
 '0x0768a75f616b98ee0937673bd83b7abf142236ea',
 False,
 True,
 0.1,
 0.5)

In [18]:
context.block_number

41455653

In [19]:
(
    vault_ichi.functions.getTotalAmounts().call(),
    vault_ichi.functions.getBasePosition().call(),
    vault_ichi.functions.getLimitPosition().call(),
    token0.balance_of(vault_ichi.address.checksum),
    token1.balance_of(vault_ichi.address.checksum),
)

# 286852599603313080740 - 26182028192636748 - 286826417575120443992
# 274646900430582100 - 274646900430582100 - 0

# totalAmount0	totalAmount1	feeAmount0	feeAmount1	totalSupply
# 274911385738111039	286846445532138665343	16019734579489	0	280732241047671948180

([664281655809637179, 277984826827539708792],
 [3193493513008405666, 662052001600269350, 56990964342029431],
 [350987158883326741955, 2229654209367829, 277927835863197679361],
 0,
 0)

In [21]:
# vault_pool()
base_lower = vault_ichi.functions.baseLower().call()
base_upper = vault_ichi.functions.baseUpper().call()

limit_lower = vault_ichi.functions.limitLower().call()
limit_upper = vault_ichi.functions.limitUpper().call()

base_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [vault_ichi.address.checksum, base_lower, base_upper])
limit_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [vault_ichi.address.checksum, limit_lower, limit_upper])

vault_pool.functions.positions(base_position_key).call(), vault_pool.functions.positions(limit_position_key).call(), 

# liquidity uint128, feeGrowthInside0LastX128 uint256, feeGrowthInside1LastX128 uint256, tokensOwed0 uint128, tokensOwed1 uint128


([3193493513008405666,
  2875935123515081046214216637017803,
  98895976994737107529842726233571982,
  26999531736089,
  928444125045983],
 [350987158883326741955,
  2160902530510430322218937526472734,
  49766990538930166157750905769655732,
  2229654209367829,
  51350386413071495])

In [22]:
# total supply
df_transfer = pd.DataFrame(vault_ichi.fetch_events(vault_ichi.events.Transfer, from_block=0))
df_transfer.loc[:, 'net_value'] = df_transfer.apply(lambda r: r.value if r['from'] == '0x0000000000000000000000000000000000000000' else -r.value, axis=1)
assert df_transfer.net_value.sum() == vault_ichi.total_supply
display(vault_ichi.total_supply_scaled)

280.63493836934117

In [23]:
# deposit
depositor = Address('0x3fabe6f79f5119336886bbbdfc0b0853dc831919')

deposit_block = 41028728
context.web3.eth.default_block = deposit_block - 1
display((deposit_block - 1, vault_ichi.scaled(vault_ichi.balance_of(Address(depositor).checksum))))

context.web3.eth.default_block = deposit_block
display((deposit_block, vault_ichi.scaled(vault_ichi.balance_of(Address(depositor).checksum))))

# context.web3.eth.default_block = fix_block_number

(41028727, 0.0)

(41028728, 278.10168836934116)

In [24]:
context.block_number

41455653

In [ ]:
tick_price0 = tick_to_price(current_tick) * scale_multiplier
if math.isclose(0, tick_price0):
    tick_price1 = 0
else:
    tick_price1 = 1/tick_price0

ratio_price0 = sqrtPriceX96 * sqrtPriceX96 / (2 ** 192) * scale_multiplier
try:
    ratio_price1 = 1/ratio_price0
except (FloatingPointError, ZeroDivisionError):
    ratio_price0 = 0
    ratio_price1 = 0


In [44]:
scale_multiplier = 10 ** (token0.decimals - token1.decimals)

current_tick = vault_ichi.functions.currentTick().call()
sqrtPriceX96 = vault_pool.functions.slot0().call()[0]

from models.credmark.protocols.dexes.uniswap.univ3_math import (
    tick_to_price, in_range, out_of_range)

tick_price0 = tick_to_price(current_tick) * scale_multiplier
ratio_price0 = sqrtPriceX96 * sqrtPriceX96 / (2 ** 192) * scale_multiplier

token0_chainlink_price = context.run_model('price.oracle-chainlink', {'base': token0.address.checksum})['price']
token1_chainlink_price = context.run_model('price.oracle-chainlink', {'base': token1.address.checksum})['price']

display((
    (tick_price0, ratio_price0, ),
    token0_chainlink_price, token1_chainlink_price, token0_chainlink_price / token1_chainlink_price
))


((22.092655883172316, 22.0942835664221),
 1905.40410858,
 80.00588147756616,
 23.81580045604872)

In [45]:
allow_token0, allow_token1

(False, True)

In [47]:
# value of ichi vault token at a block

total_supply = vault_ichi.total_supply_scaled
token0_amount, token1_amount = vault_ichi.functions.getTotalAmounts().call()
token0_amount = token0.scaled(token0_amount)
token1_amount = token0.scaled(token1_amount)

(token0_amount, token1_amount, total_supply)

if allow_token0:
    token1_in_token0_amount = token1_amount / ratio_price0
    total_amount_in_token = token0_amount + token1_in_token0_amount
else:
    token0_in_token1_amount = token0_amount * ratio_price0
    total_amount_in_token = token1_amount + token0_in_token1_amount
    
display(
    total_amount_in_token
)
# (token0_amount * context.run_model('price.oracle-chainlink', {'base': token0.address.checksum})['price'] + token1_amount * context.run_model('price.oracle-chainlink', {'base': token1.address.checksum})['price']) / total_supply


292.7431078384007

In [26]:
# burn partial position ?, burn share o
# mint position based on how much token to deposit

In [27]:
# fee
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0, argument_filters={'from': vault_ichi.address.checksum, 'to': affiliate.checksum})))
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0, argument_filters={'from': vault_ichi.address.checksum, 'to': fee_recipient.checksum})))

# TODO: deposit / withdraw by proportion
# vault token value = (vault position value - fee owned by vault (10% of current in pool) / total Supply

# Flow:
# Deposit - Withdraw - Re-balance 
# Deposit - Withdraw (In) / (vault position value - fee owned by vault (10% of current in pool)


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,412,87,b'P\xa8\x04~\x89\xc5J\x11=\x96\x19t\xe8$\x0e\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'R\x89\xb7\xcc\xb4\x8d\xed\x9c\x06A`l\xfdZ\x0...,39709682,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,146250557634778
1,"(from, to, value)",Transfer,35,10,b'\xaeI\xa6\xaa\x0bO!\xa8gJ\xca\x82FI\xe3\xc3\...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b' \n\xa9S\x0bP\x1b\tV8NA:Y\xd4Z8\x90u\xa9\xbf...,39740183,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,7151955731198
2,"(from, to, value)",Transfer,61,11,"b""\xf9'\x1d!\x01t\x01\xa5/\x02\x96\xd9\x92^\xc...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xb23\xb5k\xa2\x0b\xd3\x87\xaa\xd2a\xf1\xfac...,39773483,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,107033911253114
3,"(from, to, value)",Transfer,252,66,b'\x8dBz\x08<\x124\x03J\x960\x86\n\xfd\x88\x9d...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xd3W\x97\xce\x81kj\xad\xe0z\x88\xf4\xed\xf5...,39775221,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,887100513790587
4,"(from, to, value)",Transfer,187,66,"b""\x9b\xa0\n\xb7a\xbe\xd9\xdawK`H\x82\x93:Q\x8...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xa5|\xd2\tcwm\xdaAn4\x8bI\x92\xb3\xff\xa2\x...,39778761,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,455430885098485
...,...,...,...,...,...,...,...,...,...,...,...
62,"(from, to, value)",Transfer,64,15,b'\xcd_9\xdb\xe9\x90\xbf\\\xc2\xc8\x1d\x1f\xd8...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\n\x04\x90\xdf\xed\xfd\xfa\xbfsi\x88\xd8\xef...,41275026,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,1391213431769
63,"(from, to, value)",Transfer,50,8,b'\x92\x82\x02\x9c}\x1c\xac\x81\n\xafyQ\xd9{\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'N2X\x9c\xc4\x10\xe0S\x8a\xf5Le\xaf\x1e\x94W\...,41317592,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,23131681345805
64,"(from, to, value)",Transfer,145,7,"b""zb\xbf\x86\xd4Y\xd6C\x9a\xb5\xc7\xff`\xba\xd...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xdb7\x8b\xf4\x9f\xb0\x03\xb8\xc2\x9ec\x91I\...,41323403,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,800986728974
65,"(from, to, value)",Transfer,130,33,"b""\xd9=\xa2\xb8\x02\x91\xf9'\xb9\x1e\xbe\xd0\x...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\x7f\x9d\x8c\xe7]\xb8\xc9:\xff\xd2\xea\x8c\x...,41366445,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,197015049551019


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,411,87,b'P\xa8\x04~\x89\xc5J\x11=\x96\x19t\xe8$\x0e\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'R\x89\xb7\xcc\xb4\x8d\xed\x9c\x06A`l\xfdZ\x0...,39709682,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,146250557634777
1,"(from, to, value)",Transfer,34,10,b'\xaeI\xa6\xaa\x0bO!\xa8gJ\xca\x82FI\xe3\xc3\...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b' \n\xa9S\x0bP\x1b\tV8NA:Y\xd4Z8\x90u\xa9\xbf...,39740183,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,7151955731198
2,"(from, to, value)",Transfer,60,11,"b""\xf9'\x1d!\x01t\x01\xa5/\x02\x96\xd9\x92^\xc...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xb23\xb5k\xa2\x0b\xd3\x87\xaa\xd2a\xf1\xfac...,39773483,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,107033911253114
3,"(from, to, value)",Transfer,251,66,b'\x8dBz\x08<\x124\x03J\x960\x86\n\xfd\x88\x9d...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xd3W\x97\xce\x81kj\xad\xe0z\x88\xf4\xed\xf5...,39775221,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,887100513790587
4,"(from, to, value)",Transfer,186,66,"b""\x9b\xa0\n\xb7a\xbe\xd9\xdawK`H\x82\x93:Q\x8...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xa5|\xd2\tcwm\xdaAn4\x8bI\x92\xb3\xff\xa2\x...,39778761,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,455430885098485
...,...,...,...,...,...,...,...,...,...,...,...
62,"(from, to, value)",Transfer,63,15,b'\xcd_9\xdb\xe9\x90\xbf\\\xc2\xc8\x1d\x1f\xd8...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\n\x04\x90\xdf\xed\xfd\xfa\xbfsi\x88\xd8\xef...,41275026,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,1391213431769
63,"(from, to, value)",Transfer,49,8,b'\x92\x82\x02\x9c}\x1c\xac\x81\n\xafyQ\xd9{\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'N2X\x9c\xc4\x10\xe0S\x8a\xf5Le\xaf\x1e\x94W\...,41317592,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,23131681345805
64,"(from, to, value)",Transfer,144,7,"b""zb\xbf\x86\xd4Y\xd6C\x9a\xb5\xc7\xff`\xba\xd...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xdb7\x8b\xf4\x9f\xb0\x03\xb8\xc2\x9ec\x91I\...,41323403,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,800986728974
65,"(from, to, value)",Transfer,129,33,"b""\xd9=\xa2\xb8\x02\x91\xf9'\xb9\x1e\xbe\xd0\x...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\x7f\x9d\x8c\xe7]\xb8\xc9:\xff\xd2\xea\x8c\x...,41366445,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,197015049551018


In [28]:
# all deposit
# all withdraw
# all re-balance => fee


In [35]:
for evt in vault_ichi.abi.events:
    if evt in ['Rebalance', 'Deposit', 'Withdraw']:
        df = pd.DataFrame(vault_ichi.fetch_events(vault_ichi.events[evt], from_block=0))
        print(evt)
        display(df)

Deposit


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,sender,to,shares,amount0,amount1
0,"(sender, to, shares, amount0, amount1)",Deposit,386,65,"b'!""\xbfn\x9f\xd0~o \xe5\xfcu\xde\xa4N\x93\xd6...",0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\x030\x82\n\xed\x96\x88\x12^}v?M\xa1\x1a\xfb...,39589815,0x11111D16485aa71D2f2BfFBD294DCACbaE79c1d4,0x11111D16485aa71D2f2BfFBD294DCACbaE79c1d4,2000000000000000,0,2000000000000000
1,"(sender, to, shares, amount0, amount1)",Deposit,398,63,b'\x9c\x150\xeeDc;X\xf9hG\xfb\xe7\xf0wS\x7f\x0...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xafm\xaf\xf9\xc9\x07\xb1\\\x9dO6\x1a\x19\x0...,39596269,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,162000000000000000000,0,162000000000000000000
2,"(sender, to, shares, amount0, amount1)",Deposit,168,30,b'\x7f\x04\xb3[\xdb\xee@\xa6\xc6@\x98\xac\xf5W...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'l\xb7\xa5\xc2\xe0\xb5z\xd75\x82\xc8/KA\xe7\x...,40308621,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,289476194039859949591,0,293000000000000000000
3,"(sender, to, shares, amount0, amount1)",Deposit,246,41,b'\xe9\x14T\xb4j\x8c\x8caH\x14\xb253\xf44\xe2\...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'*\x11!3D\xd9\x045\xa2\xc5\x15\x03@\xa2\xba\x...,40870280,0xFC5A720B05DB97C4eA318F17CE8028a399EC2429,0xFC5A720B05DB97C4eA318F17CE8028a399EC2429,97302678330766235,0,100000000000000000
4,"(sender, to, shares, amount0, amount1)",Deposit,187,38,b'\xdc\xa7B\xa6W\x8e\xdb\xceI\xf0\xfb\x0e\xe2\...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\x13\xfc\x0fc\xd1\xf9\xd5[\xe0\xf9\x8d\xc8\x...,41028467,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,369078195534464927090,0,378881661360422313925
5,"(sender, to, shares, amount0, amount1)",Deposit,185,38,b'\xd1\xd3i|\xd9\x89js\xad\xf6\x95jt\x9b$\xf5_...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\n\xc3\xb3\xd5\xd4+\xc1M?\xc6\x03\xe5D\\\xb6...,41028728,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,278101688369341181945,0,290000000000000000000


Rebalance


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,tick,totalAmount0,totalAmount1,feeAmount0,feeAmount1,totalSupply
0,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,56,12,b'Zj\xce\xb00OH\xfc\xe0A\xe1\xb8}\x81\x8eE\xc1...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,"b',~\x10|\x7f\xa68\x81,tB\x0eh\xd1\xda`\x1d\xb...",39609994,29069,0,162002000000000000000,0,0,162002000000000000000
1,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,415,87,b'P\xa8\x04~\x89\xc5J\x11=\x96\x19t\xe8$\x0e\x...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'R\x89\xb7\xcc\xb4\x8d\xed\x9c\x06A`l\xfdZ\x0...,39709682,29413,2632510037426003,162049739196037827583,2925011152695558,53043551153141759,162002000000000000000
2,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,38,10,b'\xaeI\xa6\xaa\x0bO!\xa8gJ\xca\x82FI\xe3\xc3\...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b' \n\xa9S\x0bP\x1b\tV8NA:Y\xd4Z8\x90u\xa9\xbf...,39740183,29651,2730206308413506,162052774063383085391,143039114623966,2711101965758308,162002000000000000000
3,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,64,11,"b""\xf9'\x1d!\x01t\x01\xa5/\x02\x96\xd9\x92^\xc...",0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xb23\xb5k\xa2\x0b\xd3\x87\xaa\xd2a\xf1\xfac...,39773483,30125,4592873194600186,162091172458373829100,2140678225062285,41253916132248818,162002000000000000000
4,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,255,66,b'\x8dBz\x08<\x124\x03J\x960\x86\n\xfd\x88\x9d...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xd3W\x97\xce\x81kj\xad\xe0z\x88\xf4\xed\xf5...,39775221,29702,1592039899676670888,131030040270720773440,17742010275811748,36362114620038116,162002000000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,67,15,b'\xcd_9\xdb\xe9\x90\xbf\\\xc2\xc8\x1d\x1f\xd8...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\n\x04\x90\xdf\xed\xfd\xfa\xbfsi\x88\xd8\xef...,41275026,31289,276864966871910086,286781237523916056497,27824268635380,307732218777119,280732241047671948180
106,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,53,8,b'\x92\x82\x02\x9c}\x1c\xac\x81\n\xafyQ\xd9{\x...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'N2X\x9c\xc4\x10\xe0S\x8a\xf5Le\xaf\x1e\x94W\...,41317592,31578,273311014253619990,286883526090501297826,462633626916103,11400295337584851,280732241047671948180
107,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,146,7,"b""zb\xbf\x86\xd4Y\xd6C\x9a\xb5\xc7\xff`\xba\xd...",0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xdb7\x8b\xf4\x9f\xb0\x03\xb8\xc2\x9ec\x91I\...,41323403,31462,274911385738111039,286846445532138665343,16019734579489,0,280732241047671948180
108,"(tick, totalAmount0, totalAmount1, feeAmount0,...",Rebalance,133,33,"b""\xd9=\xa2\xb8\x02\x91\xf9'\xb9\x1e\xbe\xd0\x...",0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\x7f\x9d\x8c\xe7]\xb8\xc9:\xff\xd2\xea\x8c\x...,41366445,31297,668070559971357349,277892407594164681045,3940300991020378,112171967495584,280732241047671948180


Withdraw


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,sender,to,shares,amount0,amount1
0,"(sender, to, shares, amount0, amount1)",Withdraw,202,43,"b'$\xc6\x87\xb6,\x18E:T)\xd7\xc0Ui\xee\xc2\xe6...",0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xc8\x97P\xc3wN\xf5q\x02\x1bo\xe8\xb1\x8aB&....,39785437,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,121500000000000000000,2029889066493904135,82635216192174688614
1,"(sender, to, shares, amount0, amount1)",Withdraw,140,41,b'\xf6\xb2\xf8\x0b\xbb6b\xda\xf5\xe1O\xdb:\xd9...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,"b""\xa5\xe3\xda\xc8V4\xd8\xf5\xf21\xd6\xb2\x1e\...",39785942,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,20250000000000000000,338925007595334440,13807256572923668261
2,"(sender, to, shares, amount0, amount1)",Withdraw,162,39,b'+;\xb6\x1b\xda8\xcd!\xb8zjO\xa0s!#s\xb8\xb9\...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,"b'\xf4\xa9\'Y]\xe1\xfccJ&j\xa9\xb8B-}#""Ye\x1dE...",39785964,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,10125000000000000000,169462503797667219,6903628286461834130
3,"(sender, to, shares, amount0, amount1)",Withdraw,282,62,b'\xb9\xd4&t\xb2\xbfY2\xc93L4\x957\x1c\xda\xbe...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\x19\x16!y\xf4X5\x98\x86\xe5C\x0f\xe8w9kY\x9...,40016874,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,0x12C7824845440DA1B0eBfcB33E15CCDB07bceA63,7593750000000000000,122104784443795077,5513350213786174692
4,"(sender, to, shares, amount0, amount1)",Withdraw,162,34,b'\x8c\x04\xb79|\xc0\xbbA\xe2\x83\x0bi\x81\x19...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,b'\xb4jR)s\xea\x1a\xa1~\xcc \xe4\x03\xb7\xbe^F...,41028518,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,0x3faBe6F79f5119336886bbbDfc0B0853DC831919,658554389574324876681,1020262794025782739,653459180402538644116
5,"(sender, to, shares, amount0, amount1)",Withdraw,240,54,b'b^= \x91\xc4\xdaqc\x90\xd1\xcf\xfc\xbf\xa7\x...,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,"b""'\xc5Q_\xad\x7fWA\xb7\xe0\xa7\xc4\x8b\x96\xa...",41454653,0xFC5A720B05DB97C4eA318F17CE8028a399EC2429,0xFC5A720B05DB97C4eA318F17CE8028a399EC2429,97302678330766235,229539508507030,96365696561312236


In [30]:
df.loc[107, 'transactionHash']

HexBytes('0x7a62bf86d459d6439ab5c7ff60bad3660723144f4a38a8fc20b26327cdf388e3')